<a href="https://colab.research.google.com/github/YeimeLeandro/Hyperblog/blob/main/Punto_2_Examen_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

def generar_vector_unitario_p(n, p, inicial='ones'):
    """
    Genera un vector unitario respecto a la norma p.

    Parámetros:
    n (int): Dimensión del vector.
    p (float): Orden de la norma.
    inicial (str): Tipo de vector inicial ('random' o 'ones').

    Retorna:
    numpy.ndarray: Vector unitario de dimensión n.
    """
    if inicial == 'ones':
        vector = np.ones(n)
    else:
        vector = np.random.uniform(-1, 1, n)

    norma_p = np.linalg.norm(vector, ord=p)
    vector_unitario = vector / norma_p
    return vector_unitario

def metodo_aitken(normas, iteracion):
    """
    Aplica el método de Aitken para acelerar la convergencia.

    Parámetros:
    normas (list): Lista de las últimas tres normas calculadas.
    iteracion (int): Número de la iteración actual.

    Retorna:
    float: Norma extrapolada.
    """
    if len(normas) < 3:
        return normas[-1]
    else:
        n0, n1, n2 = normas[-3], normas[-2], normas[-1]
        numerator = (n2 - n1)**2
        denominator = n0 - 2*n1 + n2
        if denominator == 0:
            return n2
        else:
            aitken = n2 - numerator / denominator
            print(f"Iteración {iteracion}: Norma extrapolada (Aitken) = {aitken:.6f}")
            return aitken

def aproximar_norma_inducida():
    try:
        # Solicitar al usuario ingresar el valor de p
        p_input = input("Ingrese el valor de p para la norma vectorial (p ≥ 1 o 'inf' para norma infinito): ").strip()

        if p_input.lower() in ['inf', 'infinito', 'infinity']:
            p = np.inf
        else:
            p = float(p_input)
            if p < 1:
                print("El valor de p debe ser mayor o igual a 1.")
                return

        # Solicitar al usuario ingresar el número de filas y columnas
        filas = int(input("Ingrese el número de filas de la matriz A: "))
        columnas = int(input("Ingrese el número de columnas de la matriz A: "))

        if filas <= 0 or columnas <= 0:
            print("El número de filas y columnas debe ser mayor que cero.")
            return

        # Inicializar la matriz A
        A = []
        print("\nIngrese los elementos de la matriz A fila por fila.")
        for i in range(filas):
            while True:
                fila_input = input(f"Ingrese los elementos de la fila {i+1}, separados por espacios: ")
                elementos = fila_input.strip().split()
                if len(elementos) != columnas:
                    print(f"Debe ingresar exactamente {columnas} elementos.")
                else:
                    try:
                        fila = [float(num) for num in elementos]
                        A.append(fila)
                        break
                    except ValueError:
                        print("Por favor, ingrese números válidos.")

        A = np.array(A)

        # Verificar si la matriz es válida
        if A.shape != (filas, columnas):
            print("La matriz ingresada no es válida.")
            return

        # Parámetros del algoritmo
        tol = 1e-6
        max_iter = 1000

        # Aproximar la norma inducida
        norma_inducida, vector_unitario = aproximar_norma_inducida_matriz(A, p, tol, max_iter)

        # Mostrar los resultados
        print("\nAproximación de la norma inducida:", norma_inducida)
        print("Vector unitario asociado (x):")
        print(vector_unitario)

    except ValueError:
        print("Entrada inválida. Por favor, ingrese números válidos.")

def aproximar_norma_inducida_matriz(A, p, tol=1e-6, max_iter=1000):
    n = A.shape[1]  # Número de columnas de A
    m = A.shape[0]  # Número de filas de A

    # Paso 1: Generar un vector unitario inicial de dimensión n (puede ser 'ones' para acelerar)
    x = generar_vector_unitario_p(n, p, inicial='ones')  # Cambiar a 'random' si se desea
    print("\nVector inicial x:")
    print(x)

    # Inicializar variables
    alpha_prev = 0
    historial_normas = []

    for k in range(max_iter):
        # Paso 2: Multiplicar A por x
        y = A @ x  # y es de dimensión m

        # Paso 3: Calcular la norma p de y
        norma_y = np.linalg.norm(y, ord=p)
        if norma_y == 0:
            print("El vector y es cero. Se detiene la iteración.")
            break

        # Normalizar y
        y_unitario = y / norma_y

        # Paso 4: Multiplicar A^T por y_unitario
        x_new = A.T @ y_unitario  # x_new es de dimensión n

        # Paso 5: Calcular la norma p de x_new
        norma_x_new = np.linalg.norm(x_new, ord=p)
        if norma_x_new == 0:
            print("El vector x_new es cero. Se detiene la iteración.")
            break

        # Normalizar x_new
        x_new = x_new / norma_x_new

        # Calcular alpha actual
        alpha = norma_y

        # Añadir a historial
        historial_normas.append(alpha)

        # Imprimir la norma actual
        print(f"Iteración {k+1}: Norma = {alpha:.6f}")

        # Aplicar el método de Aitken para acelerar la convergencia
        if len(historial_normas) >= 3:
            alpha_aitken = metodo_aitken(historial_normas, k+1)
            if abs(alpha_aitken - alpha_prev) < tol:
                print(f"Convergencia alcanzada en {k+1} iteraciones (Aitken).")
                x = x_new
                alpha = alpha_aitken
                break
            else:
                alpha_prev = alpha_aitken
        else:
            # Verificar la convergencia sin Aitken
            if abs(alpha - alpha_prev) < tol:
                print(f"Convergencia alcanzada en {k+1} iteraciones.")
                x = x_new
                break
            alpha_prev = alpha

        # Preparar para la siguiente iteración
        x = x_new

    else:
        print("Se alcanzó el número máximo de iteraciones sin convergencia.")

    return alpha, x_new

# Ejecutar la función principal
aproximar_norma_inducida()

Ingrese el valor de p para la norma vectorial (p ≥ 1 o 'inf' para norma infinito): 2
Ingrese el número de filas de la matriz A: 2
Ingrese el número de columnas de la matriz A: 2

Ingrese los elementos de la matriz A fila por fila.
Ingrese los elementos de la fila 1, separados por espacios: 1 2
Ingrese los elementos de la fila 2, separados por espacios: 1 2

Vector inicial x:
[0.70710678 0.70710678]
Iteración 1: Norma = 3.000000
Iteración 2: Norma = 3.162278
Iteración 3: Norma = 3.162278
Iteración 3: Norma extrapolada (Aitken) = 3.162278
Convergencia alcanzada en 3 iteraciones (Aitken).

Aproximación de la norma inducida: 3.1622776601683795
Vector unitario asociado (x):
[0.4472136  0.89442719]
